In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Notebook description

Notebook to make preliminary plots of the HFSE abundaces measured in the CV chondrites. The notebook uses two seperate datasets collected in 2019 and 2022. All of the data was collected using the LA-ICP-MS at Rutgers University. The meteorites analyzed include:
- Vigps1B
- Vigps2B
- Leops2B
- Alps1
- Alps6
- Alps9

# Read in data

## 2022 data

In [19]:
data_2022 = pd.read_csv('2022_June_SQ_input2.csv')
data_2022

,Total_Order,Run_Order,Run_Date,Year,LA_Name,Met_Name,Met_Type,Object,Map_region,Spot_Num,...,Pb206_ppm_2SE(int),Pb206_ppm_2SD,Th232_ppm_mean,Th232_ppm_median,Th232_ppm_2SE(int),Th232_ppm_2SD,U238_ppm_mean,U238_ppm_median,U238_ppm_2SE(int),U238_ppm_2SD
0,1,1.0,6/22/22,2022,BIR-25,BIR,standard,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2.0,6/22/22,2022,BIR-25,BIR,standard,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,3.0,6/22/22,2022,BIR-25,BIR,standard,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,4.0,6/22/22,2022,Vigps1B_c14_9,Vigps1B,CV,c14,NaN,9,...,0.067496,0.527162,NaN,NaN,NaN,NaN,0.003396,-8.940000e-12,0.006508,0.050828
4,5,5.0,6/22/22,2022,Vigps1B_c14_10,Vigps1B,CV,c14,NaN,10,...,0.074675,0.578430,NaN,NaN,NaN,NaN,0.047421,2.620000e-12,0.024658,0.191000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,411,258.0,6/29/22,2022,Alps6_c5_1,Alps6,CV,c5,NaN,1,...,0.003734,0.028188,0.001339,0.000000,0.000947,0.007148,0.000000,0.000000e+00,0.000000,0.000000
408,412,259.0,6/29/22,2022,BCR-65,BCR,check_standard,NaN,NaN,2,...,0.432777,3.380094,7.437679,7.175756,0.194233,1.517005,3.189052,3.139050e+00,0.098848,0.772026
409,413,260.0,6/29/22,2022,BIR-65,BIR,standard,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,414,261.0,6/29/22,2022,BIR-65,BIR,standard,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
data_2022['Met_Name'].unique()

array(['BIR', 'Vigps1B', 'BCR', 'Vigps2B', 'Leops2B', 'Alps9', 'Alps6'],
      dtype=object)

## 2019 data

In [21]:
data_2019 = pd.read_csv('2019_LA_CVdata_SQ_input.csv')
data_2019

,Run_Order,Run_Date,Year,Book_num,Book_Order_rawdata,LA_Name,Met_Name,Met_Type,Object,Spot_Num,...,Re185_ppm_2SE(int),Re185_ppm_2SD,Th232_ppm_mean,Th232_ppm_median,Th232_ppm_2SE(int),Th232_ppm_2SD,U238_ppm_mean,U238_ppm_median,U238_ppm_2SE(int),U238_ppm_2SD
0,1,7/18/19,2019,1,newlines_1,614,614,standard,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,7/18/19,2019,1,newlines_2,612,612,standard,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,7/18/19,2019,1,newlines_3,610,610,standard,NaN,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,7/18/19,2019,1,newlines_4,BHVO,BHVO,check_standard,NaN,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,7/18/19,2019,1,newlines_5,BIR,BIR,standard,NaN,1,...,0.0,0.0,0.028000,0.000002,0.008883,0.109156,0.031991,1.120000e-07,0.010450,0.128406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,233,7/18/19,2019,13,85micron_15,BIR,BIR,standard,NaN,1,...,0.0,0.0,0.027998,0.031472,0.007421,0.058902,0.031979,3.864216e-02,0.009526,0.075612
233,234,7/18/19,2019,13,85micron_16,BCR,BCR,check_standard,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234,235,7/18/19,2019,13,85micron_17,614,614,standard,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,236,7/18/19,2019,13,85micron_18,612,612,standard,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
data_2019['Met_Name'].unique()

array(['614', '612', '610', 'BHVO', 'BIR', 'BCR', 'Alps9', 'Alps1',
       'Vigps2B', 'Vigps1B', 'Leops2B'], dtype=object)

# Data finder function

In [23]:
data_2022.groupby('Met_Name')

In [27]:
data_2022[data_2022.Met_Name == 'Alps9']

,Total_Order,Run_Order,Run_Date,Year,LA_Name,Met_Name,Met_Type,Object,Map_region,Spot_Num,...,Pb206_ppm_2SE(int),Pb206_ppm_2SD,Th232_ppm_mean,Th232_ppm_median,Th232_ppm_2SE(int),Th232_ppm_2SD,U238_ppm_mean,U238_ppm_median,U238_ppm_2SE(int),U238_ppm_2SD
150,154,4.0,6/29/22,2022,Alps9_c8b_2,Alps9,CV,c8,b,2,...,0.048634,0.376717,0.176949,1.738832e-01,0.031271,0.242224,7.224648e-02,1.110000e-07,2.723583e-02,2.109679e-01
151,155,5.0,6/29/22,2022,Alps9_c8b_3,Alps9,CV,c8,b,3,...,0.000006,0.000045,0.002879,-3.330000e-20,0.003241,0.025309,8.340000e-09,9.050000e-09,3.820000e-10,2.980000e-09
152,156,6.0,6/29/22,2022,Alps9_c8b_5,Alps9,CV,c8,b,5,...,0.077162,0.597696,0.065270,5.673536e-02,0.016597,0.128558,1.814124e-02,4.680000e-10,1.113518e-02,8.625270e-02
153,157,7.0,6/29/22,2022,Alps9_c8b_6,Alps9,CV,c8,b,6,...,0.035748,0.276905,0.044007,1.550000e-22,0.021667,0.167833,2.407280e-02,3.940000e-11,1.582962e-02,1.226157e-01
154,158,8.0,6/29/22,2022,Alps9_c9_4,Alps9,CV,c9,NaN,4,...,0.091350,0.652367,0.207948,1.945761e-01,0.030152,0.215329,1.317990e-01,1.138252e-01,3.796887e-02,2.711520e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,383,230.0,6/29/22,2022,Alps9_c26_2,Alps9,CV,c26,NaN,2,...,0.000045,0.000343,0.000000,0.000000e+00,0.000000,0.000000,NaN,NaN,NaN,NaN
396,400,247.0,6/29/22,2022,Alps9_c4b_1,Alps9,CV,c4,b,1,...,0.010683,0.080658,0.006144,0.000000e+00,0.002819,0.021283,NaN,NaN,NaN,NaN
404,408,255.0,6/29/22,2022,Alps9_c17_1,Alps9,CV,c17,NaN,1,...,0.006164,0.048142,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
405,409,256.0,6/29/22,2022,Alps9_c10_1,Alps9,CV,c10,NaN,1,...,0.000105,0.000805,0.000369,0.000000e+00,0.000513,0.003938,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [52]:
col_names = data_2022.columns.values
test_str = col_names[449]
test_str

'Cr52_ppm_mean'

In [71]:
('U' or 'ppm' or 'mean') in test_str

False

In [83]:
for x in col_names:
    if ('Cr' in x) and ('ppm' in x) and ('mean' in x):
        col_grab = x

In [84]:
col_grab

'Cr52_ppm_mean'

In [ ]:
def data_find(pd_df, by_meteorite=False, meteorite='', obj='', spot='', by_phase=False, phase='', element):
    if by_meteorite == True:
        df = pd_df[pd_df.Met_Name == 'meteorite']
    

## Marina's import code

In [ ]:
#import 2019 data
CV2019_SQ = pd.read_csv('/Users/Marina/Desktop/CV_REE_working/LA_ICPMS_data/2019_Rutgers/2019_python_input/2019_LA_CVdata_SQ_input.csv',index_col=None)
CV2019_SQ = CV2019_SQ.fillna(np.nan)
CV2019_SQ.Spot_Size = CV2019_SQ.Spot_Size.astype(int)
CV2019_SQ.Spot_Num = CV2019_SQ.Spot_Num.astype(int)
CV2019_SQ.Standard_Spot_Size = CV2019_SQ.Standard_Spot_Size.astype(int)

CV2019_Si = pd.read_csv('/Users/Marina/Desktop/CV_REE_working/LA_ICPMS_data/2019_Rutgers/2019_python_input/2019_LA_CVdata_Si_input.csv',index_col=None)
CV2019_Si = CV2019_Si.fillna(np.nan)
CV2019_Si.Spot_Size = CV2019_Si.Spot_Size.astype(int)
CV2019_Si.Spot_Num = CV2019_Si.Spot_Num.astype(int)
CV2019_Si.Standard_Spot_Size = CV2019_Si.Standard_Spot_Size.astype(int)

CV2019_Mg = pd.read_csv('/Users/Marina/Desktop/CV_REE_working/LA_ICPMS_data/2019_Rutgers/2019_python_input/2019_LA_CVdata_Mg_input.csv',index_col=None)
CV2019_Mg = CV2019_Mg.fillna(np.nan)
CV2019_Mg.Spot_Size = CV2019_Mg.Spot_Size.astype(int)
CV2019_Mg.Spot_Num = CV2019_Mg.Spot_Num.astype(int)
CV2019_Mg.Standard_Spot_Size = CV2019_Mg.Standard_Spot_Size.astype(int)

CV2019_Ca = pd.read_csv('/Users/Marina/Desktop/CV_REE_working/LA_ICPMS_data/2019_Rutgers/2019_python_input/2019_LA_CVdata_Ca_input.csv',index_col=None)
CV2019_Ca = CV2019_Ca.fillna(np.nan)
CV2019_Ca.Spot_Size = CV2019_Ca.Spot_Size.astype(int)
CV2019_Ca.Spot_Num = CV2019_Ca.Spot_Num.astype(int)
CV2019_Ca.Standard_Spot_Size = CV2019_Ca.Standard_Spot_Size.astype(int)


#import 2022 data
CV2022_SQ = pd.read_csv('/Users/Marina/Desktop/2022_REE_work/Rutgers_June2022/Rutgers_data_June2022/2022_python_input/2022_June_SQ_input2.csv',index_col=None)
CV2022_SQ = CV2022_SQ.fillna(np.nan)
CV2022_SQ.Spot_Size = CV2022_SQ.Spot_Size.astype(int)
CV2022_SQ.Spot_Num = CV2022_SQ.Spot_Num.astype(int)
CV2022_SQ.Standard_Spot_Size = CV2022_SQ.Standard_Spot_Size.astype(int)
CV2022_SQ.Met_Name = CV2022_SQ.Met_Name.astype(str)
CV2022_SQ.Met_Type = CV2022_SQ.Met_Type.astype(str)

CV2022_Si = pd.read_csv('/Users/Marina/Desktop/2022_REE_work/Rutgers_June2022/Rutgers_data_June2022/2022_python_input/2022_June_Si_input2.csv',index_col=None)
CV2022_Si = CV2022_Si.fillna(np.nan)
CV2022_Si.Spot_Size = CV2022_Si.Spot_Size.astype(int)
CV2022_Si.Spot_Num = CV2022_Si.Spot_Num.astype(int)
CV2022_Si.Standard_Spot_Size = CV2022_Si.Standard_Spot_Size.astype(int)

CV2022_Mg = pd.read_csv('/Users/Marina/Desktop/2022_REE_work/Rutgers_June2022/Rutgers_data_June2022/2022_python_input/2022_June_Mg_input2.csv',index_col=None)
CV2022_Mg = CV2022_Mg.fillna(np.nan)
CV2022_Mg.Spot_Size = CV2022_Mg.Spot_Size.astype(int)
CV2022_Mg.Spot_Num = CV2022_Mg.Spot_Num.astype(int)
CV2022_Mg.Standard_Spot_Size = CV2022_Mg.Standard_Spot_Size.astype(int)

CV2022_Ca = pd.read_csv('/Users/Marina/Desktop/2022_REE_work/Rutgers_June2022/Rutgers_data_June2022/2022_python_input/2022_June_Ca_input2.csv',index_col=None)
CV2022_Ca = CV2022_Ca.fillna(np.nan)
CV2022_Ca.Spot_Size = CV2022_Ca.Spot_Size.astype(int)
CV2022_Ca.Spot_Num = CV2022_Ca.Spot_Num.astype(int)
CV2022_Ca.Standard_Spot_Size = CV2022_Ca.Standard_Spot_Size.astype(int)

CV2022_Al = pd.read_csv('/Users/Marina/Desktop/2022_REE_work/Rutgers_June2022/Rutgers_data_June2022/2022_python_input/2022_June_Al_input2.csv',index_col=None)
CV2022_Al = CV2022_Al.fillna(np.nan)
CV2022_Al.Spot_Size = CV2022_Al.Spot_Size.astype(int)
CV2022_Al.Spot_Num = CV2022_Al.Spot_Num.astype(int)
CV2022_Al.Standard_Spot_Size = CV2022_Al.Standard_Spot_Size.astype(int)
